In [3]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
from docx import Document
import re
from jinja2 import Environment, FileSystemLoader
import pypandoc

In [18]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")
PROTOCOL = 139

URL = f"{BASE_URL}/{PROTOCOL}"

In [19]:
headers = {"Authorization": f"Bearer {API_KEY}"}
response = requests.get(URL, headers=headers)

In [ ]:
if response.status_code == 200:
    print("Data fetched successfully:")
    print(response.json())  # Print the JSON response
else:
    print(f"Failed to fetch data: {response.status_code} - {response.text}")

In [24]:
vesgitio_list = response.json()["listaVestigio"]

In [ ]:
with open("resp.json", "w") as json_file:
    json.dump(response.json(), json_file, indent=4)
    print(f"Data saved")

In [ ]:
doc = Document()

doc.add_heading(f"Protocolo número: {PROTOCOL}", level=0)

doc.add_heading("Vestigios", level=1)

for i, v, in enumerate(vesgitio_list):
    doc.add_heading(f"Vestigio ID: {v["id"]}", level=2)
    doc.add_paragraph(v['resumo'], style="ListBullet")
    
    doc.add_heading(f"Tabela de dados {str(i+1)}", level=3)
    
    table_data = (v['dadosVestigio'])
    table = doc.add_table(rows=len(table_data), cols=2)
    
    for i, (header, value) in enumerate(table_data.items(), start=0):
      
      if(value != None):
        cell = table.cell(i, 0)
        cell.text = str(camel_case_to_capitalized(header))
        cell = table.cell(i, 1)
        cell.text = str(value)

delete_empty_rows(table)

In [84]:
doc.save("my_document.docx")

In [88]:
df = pd.DataFrame(vesgitio_list)

In [95]:
df = pd.json_normalize(response.json()['solicitacaoOcorrencia'])

In [98]:
df.to_excel('output.ods', engine='odf')

In [102]:
pd.DataFrame(response.json()['solicitacaoOcorrencia'])

df['dataInclusao'] = pd.to_datetime(df['dataInclusao'], unit='ms').dt.strftime('%d/%m/%Y %H:%M:%S')
df['dataAcionamento'] = pd.to_datetime(df['dataAcionamento'], unit='ms').dt.strftime('%d/%m/%Y %H:%M:%S')
df['dataHoraOcorrencia'] = pd.to_datetime(df['dataHoraOcorrencia'], unit='ms').dt.strftime('%d/%m/%Y %H:%M:%S')
df['dataHoraAtendimentoPerito'] = pd.to_datetime(df['dataHoraAtendimentoPerito'], unit='ms').dt.strftime('%d/%m/%Y %H:%M:%S')
df['dataHoraLiberacaoLocal'] = pd.to_datetime(df['dataHoraLiberacaoLocal'], unit='ms').dt.strftime('%d/%m/%Y %H:%M:%S')

In [6]:
try:
    load_dotenv()

    API_KEY = os.getenv("API_KEY")
    BASE_URL = os.getenv("BASE_URL")
    PROTOCOL = 139

    URL = f"{BASE_URL}/{PROTOCOL}"
except Exception as e:
    logger.error(e)
    exit(1)

In [15]:
API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")
PROTOCOL = 139

In [ ]:
try:
    headers = {"Authorization": f"Bearer {API_KEY}"}
    response = requests.get(URL, headers=headers)

    response.raise_for_status()
    if response.status_code == 200:
        print("Dados recebidos com sucesso.")
    else:
        raise (f"Failed to fetch data: {response.status_code} - {response.text}")

    vesgitio_list = response.json()["listaVestigio"]
    solicitacao = response.json()["solicitacaoOcorrencia"]
except Exception as e:
    print(e)

In [ ]:
vesgitio_list

In [ ]:
env = Environment(loader=FileSystemLoader('./templates'))  # 'templates' folder contains your .html template
template = env.get_template('template.html.j2')

rendered_html = template.render(data)
output_file = 'output.odt'

pypandoc.convert_text(rendered_html, to='odt', format='html', outputfile=output_file)

print(f"Conversion completed. The DOCX file is saved as '{output_file}'.")